# Tutorial 2: Benchmarks

In [ ]:
import warnings
import sys

warnings.filterwarnings("ignore")

from sklearn.datasets import load_iris
from synthcity.plugins import Plugins
import synthcity.logger as log
from synthcity.plugins.core.dataloader import GenericDataLoader

X, y = load_iris(return_X_y=True, as_frame=True)
X["target"] = y

loader = GenericDataLoader(X, target_column="target", sensitive_columns=[])

loader.dataframe()

## List the available generative models

In [ ]:
from synthcity.plugins import Plugins

plugins = Plugins().list()

plugins

## Benchmark the quality of plugins

In [ ]:
from synthcity.benchmark import Benchmarks

score = Benchmarks.evaluate(
    [("uniform_sampler", "uniform_sampler", {})],
    loader,
    synthetic_size=len(X),
    repeats=1,
)

In [ ]:
Benchmarks.print(score)

In [ ]:
import pandas as pd
import numpy as np

means = []
directions = None
for plugin in score:
    data = score[plugin]["mean"]
    if directions is None and len(score[plugin]["direction"].to_dict()) > 0:
        directions = score[plugin]["direction"].to_dict()

    means.append(data)

out = pd.concat(means, axis=1)
out.set_axis(score.keys(), axis=1, inplace=True)

bad_highlight = "background-color: lightcoral;"
ok_highlight = "background-color: green;"
default = ""


def highlights(row):
    metric = row.name
    if directions[metric] == "minimize":
        best_val = np.min(row.values)
        worst_val = np.max(row)
    else:
        best_val = np.max(row.values)
        worst_val = np.min(row)

    styles = []
    for val in row.values:
        if val == best_val:
            styles.append(ok_highlight)
        elif val == worst_val:
            styles.append(bad_highlight)
        else:
            styles.append(default)

    return styles


out.style.apply(highlights, axis=1)

# 